# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [63]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math

## Load the Data

Start by importing the data from the pickle file.

In [64]:
pwd

'/Users/tz/Documents/carnd/misc/l11 keras'

In [65]:
# TODO: Implement load the data here.
with open('train.p', 'rb') as f:
    data = pickle.load(f)
with open('test.p', 'rb') as f:
    test = pickle.load(f)

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [66]:
print(data.keys())

dict_keys(['features', 'labels'])


In [67]:
# TODO: Use `train_test_split` here.

X_train, X_val, y_train, y_val = train_test_split(data['features'], data['labels'], 
                                                  test_size=0.33)

In [68]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [69]:
# TODO: Implement data normalization here.

X_train[0].shape

(32, 32, 3)

In [70]:
tz = np.array([23,24,23,22,1], dtype=np.float32)
from sklearn.preprocessing import normalize
print(normalize(tz.reshape(-1,1), axis=0))

tz0 = (tz-np.mean(tz))/(2*np.mean(tz))
print(tz0)

[[ 0.49964595]
 [ 0.5213697 ]
 [ 0.49964595]
 [ 0.4779222 ]
 [ 0.02172374]]
[ 0.11827955  0.14516127  0.11827955  0.09139784 -0.47311828]


In [71]:
X_train = (X_train-255./2)/(255.)
X_val = (X_val-255./2)/(255.)

In [72]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [77]:
# TODO: Build a two-layer feedforward neural network with Keras here.

'''
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])
'''
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

In [83]:
model = Sequential()
model.add(Dense(128, input_shape=(3072, )))
model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(Dense(43, input_shape=(128, )))
model.add(Activation('softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_18 (Dense)                 (None, 128)           393344      dense_input_8[0][0]              
____________________________________________________________________________________________________
activation_16 (Activation)       (None, 128)           0           dense_18[0][0]                   
____________________________________________________________________________________________________
dense_19 (Dense)                 (None, 43)            5547        activation_16[0][0]              
____________________________________________________________________________________________________
activation_17 (Activation)       (None, 43)            0           dense_19[0][0]                   
Total params: 398891
______________________________________________________________________

In [84]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3072))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

In [85]:
# Debugging
for l in model.layers:
    print(l.name, l.input_shape, l.output_shape, l.activation)

dense_18 (None, 3072) (None, 128) <function linear at 0x13aa671e0>
activation_16 (None, 128) (None, 128) <function relu at 0x13aa5cf28>
dense_19 (None, 128) (None, 43) <function linear at 0x13aa671e0>
activation_17 (None, 43) (None, 43) <function softmax at 0x13aa5cd08>


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [86]:
X_train.reshape(-1, 32*32*3).shape

(26270, 3072)

In [87]:
def to_categorical(y, nb_classes=None):
    '''Convert class vector (integers from 0 to nb_classes) to binary class matrix, 
    for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
        nb_classes: total number of classes
    # Returns
        A binary matrix representation of the input.
        
    source : https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py
    '''
    y = np.array(y, dtype='int')
    if not nb_classes:
        nb_classes = np.max(y)+1
    Y = np.zeros((len(y), nb_classes))
    for i in range(len(y)):
        Y[i, y[i]] = 1.
    return Y

In [99]:
# TODO: Compile and train the model here.
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# increasing the batch_size to 1024 increased my validation and test accuracy by a lot!
history = model.fit(X_train.reshape(-1, 32*32*3), to_categorical(y_train),
                    batch_size=1024, nb_epoch=2,
                    verbose=1, validation_data=(X_val.reshape(-1,32*32*3), to_categorical(y_val)))

Train on 26270 samples, validate on 12939 samples
Epoch 1/2
26270/26270 [==============================] - 1s - loss: 0.3988 - acc: 0.9044 - val_loss: 0.3849 - val_acc: 0.9068
Epoch 2/2
26270/26270 [==============================] - 1s - loss: 0.3218 - acc: 0.9345 - val_loss: 0.3578 - val_acc: 0.9182


In [100]:
X_test = test['features']
y_test = test['labels']

score = model.evaluate(X_test.reshape(-1, 32*32*3), to_categorical(y_test), verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 7.03699107906
Test accuracy: 0.559778305617


In [101]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1]
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here) 91.815%

In [104]:
print(history.history['acc'])
print(history.history['val_acc'])

[0.90437761733048216, 0.93452607559803824]
[0.90679341666581814, 0.91815441692987487]


## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [116]:
# from keras import backend as K
# K.image_dim_ordering()


# img_rows, img_cols = 32, 32
# if K.image_dim_ordering() == 'th':
#     X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
#     X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
#     X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)
# print(input_shape)

In [117]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.

# TODO: Compile and train the model here.

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

nb_filters = 32
# pool_size = (2, 2)
kernel_size = (3, 3)
input_shape = (32,32,3)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=pool_size))
# model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, input_shape=(3072, )))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(43, input_shape=(128, )))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(), #optimizer='adadelta',
              metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_18 (Activation)       (None, 30, 30, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 28800)         0           activation_18[0][0]              
____________________________________________________________________________________________________
dense_20 (Dense)                 (None, 128)           3686528     flatten_1[0][0]                  
___________________________________________________________________________________________

In [134]:
history = model.fit(X_train, to_categorical(y_train), batch_size=3072, nb_epoch=10,
                    verbose=1, validation_data=(X_val, to_categorical(y_val)))

Train on 26270 samples, validate on 12939 samples
Epoch 1/10
26270/26270 [==============================] - 18s - loss: 0.7282 - acc: 0.8022 - val_loss: 0.5000 - val_acc: 0.8833
Epoch 2/10
26270/26270 [==============================] - 18s - loss: 0.6719 - acc: 0.8151 - val_loss: 0.4511 - val_acc: 0.9055
Epoch 3/10
26270/26270 [==============================] - 18s - loss: 0.6180 - acc: 0.8308 - val_loss: 0.4061 - val_acc: 0.9165
Epoch 4/10
26270/26270 [==============================] - 18s - loss: 0.5844 - acc: 0.8380 - val_loss: 0.3807 - val_acc: 0.9162
Epoch 5/10
26270/26270 [==============================] - 18s - loss: 0.5350 - acc: 0.8540 - val_loss: 0.3491 - val_acc: 0.9311
Epoch 6/10
26270/26270 [==============================] - 18s - loss: 0.4976 - acc: 0.8688 - val_loss: 0.3162 - val_acc: 0.9352
Epoch 7/10
26270/26270 [==============================] - 18s - loss: 0.4646 - acc: 0.8762 - val_loss: 0.2931 - val_acc: 0.9396
Epoch 8/10
26270/26270 [==============================

In [135]:
print(history.history['val_acc'])

[0.88329855583948458, 0.90547954298187905, 0.91653142249310571, 0.91622227987280147, 0.93106112918526507, 0.9352345714263951, 0.93963984446157411, 0.94466341916906182, 0.94713657788989325, 0.95107812685781856]


In [136]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

In [139]:
score = model.evaluate(X_test, to_categorical(y_test), verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 4.25486210727
Test accuracy: 0.731037212985


**Validation Accuracy**: (fill in here) 95.107%

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [140]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.

# TODO: Compile and train the model here.

from keras.layers import Convolution2D, MaxPooling2D

pool_size = (2, 2)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
# model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, input_shape=(3072, )))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(43, input_shape=(128, )))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(), #optimizer='adadelta',
              metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_2 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_21 (Activation)       (None, 30, 30, 32)    0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 15, 15, 32)    0           activation_21[0][0]              
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 7200)          0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [147]:
history = model.fit(X_train, to_categorical(y_train), batch_size=3072, nb_epoch=3,
                    verbose=1, validation_data=(X_val, to_categorical(y_val)))

Train on 26270 samples, validate on 12939 samples
Epoch 1/3
26270/26270 [==============================] - 16s - loss: 0.4147 - acc: 0.8866 - val_loss: 0.2549 - val_acc: 0.9490
Epoch 2/3
26270/26270 [==============================] - 16s - loss: 0.3974 - acc: 0.8910 - val_loss: 0.2445 - val_acc: 0.9487
Epoch 3/3
26270/26270 [==============================] - 16s - loss: 0.3854 - acc: 0.8959 - val_loss: 0.2338 - val_acc: 0.9536


In [148]:
print(history.history['val_acc'])

[0.94899143529773011, 0.9486822580174562, 0.95355126842832227]


In [149]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here) 95.36%

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [150]:
# TODO: Re-construct the network and add dropout after the pooling layer.

# TODO: Compile and train the model here.

from keras.layers import Convolution2D, MaxPooling2D

pool_size = (2, 2)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, input_shape=(3072, )))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(43, input_shape=(128, )))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(), 
              metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_3 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
activation_24 (Activation)       (None, 30, 30, 32)    0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 15, 15, 32)    0           activation_24[0][0]              
____________________________________________________________________________________________________
dropout_13 (Dropout)             (None, 15, 15, 32)    0           maxpooling2d_2[0][0]             
___________________________________________________________________________________________

In [151]:
history = model.fit(X_train, to_categorical(y_train), batch_size=3072, nb_epoch=30,
                    verbose=1, validation_data=(X_val, to_categorical(y_val)))

Train on 26270 samples, validate on 12939 samples
Epoch 1/30
26270/26270 [==============================] - 22s - loss: 3.4784 - acc: 0.0947 - val_loss: 3.1364 - val_acc: 0.2523
Epoch 2/30
26270/26270 [==============================] - 22s - loss: 3.0402 - acc: 0.2334 - val_loss: 2.6633 - val_acc: 0.3823
Epoch 3/30
26270/26270 [==============================] - 19s - loss: 2.6361 - acc: 0.3297 - val_loss: 2.2626 - val_acc: 0.4568
Epoch 4/30
26270/26270 [==============================] - 19s - loss: 2.2977 - acc: 0.4031 - val_loss: 1.9340 - val_acc: 0.5186
Epoch 5/30
26270/26270 [==============================] - 19s - loss: 2.0235 - acc: 0.4579 - val_loss: 1.6645 - val_acc: 0.5827
Epoch 6/30
26270/26270 [==============================] - 19s - loss: 1.7975 - acc: 0.5089 - val_loss: 1.4308 - val_acc: 0.6582
Epoch 7/30
26270/26270 [==============================] - 19s - loss: 1.5916 - acc: 0.5622 - val_loss: 1.2388 - val_acc: 0.7133
Epoch 8/30
26270/26270 [==============================

In [153]:
print(history.history['val_acc'])

[0.2522606163173603, 0.38225519716559997, 0.45675863618920359, 0.51858721729113166, 0.58273436062412809, 0.65816523128835003, 0.71334725094593343, 0.74487983061049501, 0.78545482155067747, 0.81103641219020806, 0.82896669887208396, 0.84558312122600143, 0.86490455624290496, 0.88515340881117699, 0.89589613826706604, 0.9036247016464285, 0.91104412912189225, 0.92140042823289126, 0.9256511328946182, 0.93075198884522203, 0.93129299056245307, 0.94002627144826145, 0.94103100059094902, 0.94589999685037618, 0.95386041740571814, 0.95038257391746317, 0.95015071038439192, 0.95679728496442285, 0.95726098852314589, 0.95919312693916281]


In [152]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here) 95.92%

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [155]:
# feeling lazy here
from keras.layers import Conv2D

**Best Validation Accuracy:** (fill in here) 95.92%

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [154]:
# TODO: Load test data
    
# TODO: Preprocess data & one-hot encode the labels

# TODO: Evaluate model on test data

score = model.evaluate(X_test, to_categorical(y_test), verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 3.13781832709
Test accuracy: 0.800633412463


**Test Accuracy:** (fill in here) 80%

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.